# 绝对估值法

## 📋 概述

绝对估值法（Absolute Valuation）是基于公司内在价值进行估值的分析方法，主要通过预测公司未来的现金流并折现到当前时点来确定公司的价值。绝对估值法主要包括现金流折现模型（DCF）、股利折现模型（DDM）等。本章节将详细介绍绝对估值法的原理、计算方法和实际应用。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际数据计算和可视化，深入理解绝对估值法。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib seaborn

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print("环境准备完成！")


## 📖 核心内容

### 1. 绝对估值法原理

#### 1.1 基本理念

**核心假设**：
- 公司的价值等于其未来现金流的现值
- 现金流可以预测
- 折现率可以确定

**估值公式**：
$$V = \sum_{t=1}^{n} \frac{CF_t}{(1+r)^t} + \frac{TV}{(1+r)^n}$$

其中：
- $V$：公司价值
- $CF_t$：第t期的现金流
- $r$：折现率
- $TV$：终值（Terminal Value）
- $n$：预测期数

#### 1.2 主要方法

**现金流折现模型（DCF）**：
- 自由现金流折现模型（FCF）
- 股权自由现金流折现模型（FCFE）
- 企业自由现金流折现模型（FCFF）

**股利折现模型（DDM）**：
- 固定股利模型
- 固定增长模型（Gordon模型）
- 多阶段增长模型

让我们先实现DCF模型：


In [ ]:
# DCF估值模型
class DCFValuation:
    """现金流折现估值模型"""
    
    def __init__(self, free_cash_flows, discount_rate, terminal_growth_rate, years=10):
        """
        初始化DCF模型
        
        参数:
        free_cash_flows: 自由现金流列表（未来预测值）
        discount_rate: 折现率（WACC）
        terminal_growth_rate: 终值增长率
        years: 预测年数
        """
        self.fcf = free_cash_flows
        self.r = discount_rate
        self.g = terminal_growth_rate
        self.years = years
    
    def calculate_pv(self):
        """计算现值"""
        pv_list = []
        for i, fcf in enumerate(self.fcf):
            pv = fcf / ((1 + self.r) ** (i + 1))
            pv_list.append(pv)
        return pv_list
    
    def calculate_terminal_value(self):
        """计算终值"""
        # 使用永续增长模型：TV = FCF_n * (1+g) / (r-g)
        last_fcf = self.fcf[-1]
        terminal_value = last_fcf * (1 + self.g) / (self.r - self.g)
        return terminal_value
    
    def calculate_terminal_value_pv(self):
        """计算终值现值"""
        tv = self.calculate_terminal_value()
        tv_pv = tv / ((1 + self.r) ** self.years)
        return tv_pv
    
    def calculate_enterprise_value(self):
        """计算企业价值"""
        pv_list = self.calculate_pv()
        tv_pv = self.calculate_terminal_value_pv()
        enterprise_value = sum(pv_list) + tv_pv
        return enterprise_value
    
    def calculate_equity_value(self, net_debt):
        """计算股权价值"""
        ev = self.calculate_enterprise_value()
        equity_value = ev - net_debt
        return equity_value

# 示例：DCF估值
# 假设未来10年的自由现金流预测（单位：万元）
free_cash_flows = [1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800]

# 折现率（WACC）：假设为10%
discount_rate = 0.10

# 终值增长率：假设为3%
terminal_growth_rate = 0.03

# 创建DCF模型
dcf_model = DCFValuation(free_cash_flows, discount_rate, terminal_growth_rate, years=10)

# 计算现值
pv_list = dcf_model.calculate_pv()
terminal_value = dcf_model.calculate_terminal_value()
terminal_value_pv = dcf_model.calculate_terminal_value_pv()
enterprise_value = dcf_model.calculate_enterprise_value()

# 假设净债务为5000万元
net_debt = 5000
equity_value = dcf_model.calculate_equity_value(net_debt)

# 假设股本为1000万股
shares = 10000000
price_per_share = equity_value / shares

print("DCF估值结果：")
print(f"企业价值（EV）：{enterprise_value:,.0f} 万元")
print(f"终值：{terminal_value:,.0f} 万元")
print(f"终值现值：{terminal_value_pv:,.0f} 万元")
print(f"股权价值：{equity_value:,.0f} 万元")
print(f"每股价值：{price_per_share:.2f} 元")


In [ ]:
# 可视化DCF模型
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

years = list(range(1, 11))

# 1. 自由现金流预测
axes[0, 0].bar(years, free_cash_flows, color='skyblue', alpha=0.7)
axes[0, 0].set_title('自由现金流预测', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('年份')
axes[0, 0].set_ylabel('自由现金流（万元）')
axes[0, 0].grid(True, alpha=0.3)

# 2. 现值计算
axes[0, 1].bar(years, pv_list, color='lightgreen', alpha=0.7)
axes[0, 1].set_title('现金流现值', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('年份')
axes[0, 1].set_ylabel('现值（万元）')
axes[0, 1].grid(True, alpha=0.3)

# 3. 累计现值
cumulative_pv = np.cumsum(pv_list)
axes[1, 0].plot(years, cumulative_pv, marker='o', linewidth=2, markersize=8, color='coral')
axes[1, 0].axhline(y=enterprise_value - terminal_value_pv, color='red', 
                   linestyle='--', label='预测期现值')
axes[1, 0].set_title('累计现值', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('年份')
axes[1, 0].set_ylabel('累计现值（万元）')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. 价值构成
value_components = {
    '预测期现值': sum(pv_list),
    '终值现值': terminal_value_pv
}
axes[1, 1].bar(value_components.keys(), value_components.values(), 
               color=['skyblue', 'lightgreen'], alpha=0.7)
axes[1, 1].set_title('企业价值构成', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('价值（万元）')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n价值构成分析：")
print(f"预测期现值：{sum(pv_list):,.0f} 万元 ({sum(pv_list)/enterprise_value*100:.1f}%)")
print(f"终值现值：{terminal_value_pv:,.0f} 万元 ({terminal_value_pv/enterprise_value*100:.1f}%)")
print(f"企业价值：{enterprise_value:,.0f} 万元")


### 3. 股利折现模型（DDM）

股利折现模型是另一种绝对估值方法，适用于稳定分红的公司。

#### 3.1 Gordon增长模型

**公式**：
$$P_0 = \frac{D_1}{r - g} = \frac{D_0(1+g)}{r - g}$$

其中：
- $P_0$：股票当前价值
- $D_1$：下一期股利
- $D_0$：当前股利
- $r$：必要收益率
- $g$：股利增长率

让我们实现DDM模型：


In [ ]:
# Gordon增长模型（固定增长DDM）
def gordon_growth_model(dividend_0, growth_rate, required_return):
    """
    Gordon增长模型
    
    参数:
    dividend_0: 当前股利
    growth_rate: 股利增长率
    required_return: 必要收益率
    """
    if required_return <= growth_rate:
        return None, "必要收益率必须大于增长率"
    
    dividend_1 = dividend_0 * (1 + growth_rate)
    stock_value = dividend_1 / (required_return - growth_rate)
    
    return stock_value, None

# 示例：DDM估值
current_dividend = 1.0  # 当前每股股利：1元
dividend_growth_rate = 0.05  # 股利增长率：5%
required_return = 0.10  # 必要收益率：10%

stock_value, error = gordon_growth_model(current_dividend, dividend_growth_rate, required_return)

if error:
    print(f"错误：{error}")
else:
    print("DDM估值结果（Gordon增长模型）：")
    print(f"当前股利：{current_dividend:.2f} 元/股")
    print(f"下一期股利：{current_dividend * (1 + dividend_growth_rate):.2f} 元/股")
    print(f"股利增长率：{dividend_growth_rate*100:.1f}%")
    print(f"必要收益率：{required_return*100:.1f}%")
    print(f"股票价值：{stock_value:.2f} 元/股")
    
    # 可视化不同增长率下的股票价值
    growth_rates = np.linspace(0, 0.08, 50)
    stock_values = []
    for g in growth_rates:
        if required_return > g:
            val, _ = gordon_growth_model(current_dividend, g, required_return)
            stock_values.append(val)
        else:
            stock_values.append(None)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    valid_indices = [i for i, v in enumerate(stock_values) if v is not None]
    valid_growth = [growth_rates[i] for i in valid_indices]
    valid_values = [stock_values[i] for i in valid_indices]
    
    ax.plot(valid_growth, valid_values, linewidth=2, color='blue')
    ax.axvline(x=dividend_growth_rate, color='red', linestyle='--', 
               label=f'当前增长率({dividend_growth_rate*100:.1f}%)')
    ax.axhline(y=stock_value, color='red', linestyle='--', 
               label=f'当前价值({stock_value:.2f}元)')
    ax.set_title('Gordon增长模型：不同增长率下的股票价值', fontsize=14, fontweight='bold')
    ax.set_xlabel('股利增长率')
    ax.set_ylabel('股票价值（元/股）')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


### 4. 敏感性分析

估值结果对关键参数的敏感性分析非常重要，可以帮助我们了解估值的可靠性。

让我们进行敏感性分析：


In [ ]:
# DCF模型敏感性分析
def sensitivity_analysis_dcf(base_fcf, base_r, base_g):
    """DCF模型敏感性分析"""
    # 折现率范围：8% - 12%
    discount_rates = np.linspace(0.08, 0.12, 20)
    # 终值增长率范围：1% - 5%
    growth_rates = np.linspace(0.01, 0.05, 20)
    
    # 创建敏感性分析矩阵
    sensitivity_matrix = np.zeros((len(discount_rates), len(growth_rates)))
    
    for i, r in enumerate(discount_rates):
        for j, g in enumerate(growth_rates):
            if r > g:
                # 假设未来10年现金流以base_fcf为基础，每年增长5%
                fcf_forecast = [base_fcf * (1.05 ** t) for t in range(1, 11)]
                model = DCFValuation(fcf_forecast, r, g, years=10)
                ev = model.calculate_enterprise_value()
                sensitivity_matrix[i, j] = ev
    
    return discount_rates, growth_rates, sensitivity_matrix

# 进行敏感性分析
base_fcf = 1000  # 基础自由现金流
base_r = 0.10  # 基础折现率
base_g = 0.03  # 基础增长率

discount_rates, growth_rates, sensitivity_matrix = sensitivity_analysis_dcf(base_fcf, base_r, base_g)

# 可视化敏感性分析
fig, ax = plt.subplots(figsize=(12, 8))
im = ax.imshow(sensitivity_matrix, aspect='auto', cmap='RdYlGn', origin='lower')
ax.set_xticks(range(0, len(growth_rates), 4))
ax.set_xticklabels([f'{g*100:.1f}%' for g in growth_rates[::4]])
ax.set_yticks(range(0, len(discount_rates), 4))
ax.set_yticklabels([f'{r*100:.1f}%' for r in discount_rates[::4]])
ax.set_xlabel('终值增长率', fontsize=12)
ax.set_ylabel('折现率', fontsize=12)
ax.set_title('DCF模型敏感性分析（企业价值）', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax, label='企业价值（万元）')
plt.tight_layout()
plt.show()

# 找出当前参数对应的估值
current_r_idx = np.argmin(np.abs(discount_rates - base_r))
current_g_idx = np.argmin(np.abs(growth_rates - base_g))
current_value = sensitivity_matrix[current_r_idx, current_g_idx]

print(f"\n敏感性分析结果：")
print(f"当前参数（折现率={base_r*100:.1f}%, 增长率={base_g*100:.1f}%）对应的企业价值：{current_value:,.0f} 万元")
print(f"估值范围：{sensitivity_matrix.min():,.0f} - {sensitivity_matrix.max():,.0f} 万元")
print(f"估值波动幅度：{(sensitivity_matrix.max() - sensitivity_matrix.min())/current_value*100:.1f}%")


## 💡 关键要点总结

1. **绝对估值法核心**：基于未来现金流折现确定公司价值
2. **主要方法**：DCF模型、DDM模型
3. **关键参数**：折现率（WACC）、增长率、现金流预测
4. **注意事项**：参数敏感性高，需要合理假设和敏感性分析

## 🛠️ 工具与软件

### 数据获取

- **公司年报**：获取历史财务数据
- **财务预测**：需要自己预测或使用分析师预测
- **市场数据**：获取无风险利率、市场风险溢价等

### 分析工具

- **Excel**：DCF模型计算
- **Python**：批量计算、敏感性分析（本Notebook）
- **专业软件**：Bloomberg、Wind等

### 注意事项

- ✅ **参数合理性**：折现率和增长率要合理
- ✅ **现金流预测**：要基于公司实际情况预测
- ✅ **敏感性分析**：必须进行敏感性分析
- ⚠️ **假设风险**：估值结果高度依赖假设
- ⚠️ **适用性**：适合稳定增长的公司

## 🔗 相关知识点

- [相对估值法](./相对估值法.md)
- [财务比率分析](../01_财务报表分析/财务比率分析.ipynb)
- [现金流量表分析](../01_财务报表分析/现金流量表分析.md)

## 📚 拓展阅读

- 《估值：难点、解决方案及相关案例》- 估值经典教材
- 《公司财务原理》- 深入理解估值理论
- 各公司估值报告 - 实际案例分析
